In [6]:

from scipy import *
from numpy import *
#from scipy.misc import derivative
import numdifftools as nd
from scipy.special import *
from tokenize import TokenError
import bqplot.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Latex, HTML, Math
import sympy as sym
from sympy.parsing.sympy_parser import parse_expr

In [ ]:
x = sym.Symbol('x')

## Derivative Explorer
Use the entry widgets below the plots to examine the connection between the slope of a given function and the value of its derivative.

The function can be changed by typing a new function into the 'Enter function' box hitting the Return/Enter key.  The function must be described in terms of the variable 'x'. You must use an '\*' to indicate multiplication.  Powers use '\*\*'.

The domain can be changed similarly.

The position at which the derivative is calculated can be moved back and forth using the labelled slider widget.

### Try:
(1) Enter the function 'sin(3\*x) \* exp(-x/2)'

(2) Try tanh(x)



In [9]:
def set_function(strFunc):
    try:
        local_func = lambda x: eval(strFunc)
        def local_deriv(x):
            #return around(derivative(local_func, x, dx=1e-6), 4)
            return around(nd.Derivative(local_func, n=1)(x), 4)
        def local_second(x):
            #return around(derivative(local_func, x, n=2, dx=1e-4), 4)
            return around(nd.Derivative(local_func, n=2)(x), 4)
        f = parse_expr(strFunc)
    except TokenError:
        print("malformed function")
        return None
    return local_func, local_deriv, local_second, f

strFunc = "1+x**4"
f, df, d2f, fstr = set_function(strFunc)
sym_func = sym.latex(fstr)


label_layout = widgets.Layout(display='flex', justify_content='center', width = "70%", 
                              align_items = 'center', border = 'none')
fig_layout = widgets.Layout(height = "350px", width = "350px")
toggle_layout = widgets.Layout(height = "350px", width = "350px", visibility = "hidden")
fig_margin_dict = dict(top=30, bottom = 30, left = 20, right = 10)

label = widgets.Label(r"$f(x) =  {}$".format(sym.latex(1+x**2)), layout = label_layout)
title_format = "<h2><font color='blue'>$f(x) = {}$</font></h2>"
title_label = widgets.HTMLMath(title_format.format(sym_func),
                              layout = label_layout)

# create initial plots
low = -3
high = 3
xpoint = 0
nPts = 200  # number of points in domain
xx = linspace(low, high, nPts)
yy = f(xx)

# Main figure
main_fig = plt.figure(title="function", layout = fig_layout, fig_margin = fig_margin_dict)
function_curve = plt.plot(xx, yy)
ylimits = plt.ylim(min(yy), max(yy))
ypoint = f(xpoint)
point = plt.plot([xpoint], [ypoint], marker='circle', colors = ['red'])

# add derivative line to main figure:
m = df(xpoint)
b = ypoint - xpoint * m
deriv_line = lambda x: m*x + b
df_line = plt.plot(xx, deriv_line(xx), colors=['green'])

# add osculating parabola to main figure:
m2 = d2f(xpoint)
a = 0.5*m2
b = m - xpoint*m2
c = ypoint - xpoint*m + 0.5*xpoint**2*m2
osc_par = lambda x: a*x**2 + b*x + c
osc_curve = plt.plot(xx, osc_par(xx), colors=['magenta'])

# create point slider and checkboxes
point_slider = widgets.FloatSlider(value = 0.5, min = low, max = high, step = 0.01, 
                                  readout = True, readout_format = '.2f', continuous_update = True,
                                  style={'handle_color':'lightblue'})
point_label = widgets.Label("Position of red dot: ")
show_deriv = widgets.Checkbox(value = True, description = "Show first derivative" )
show_second = widgets.Checkbox(value = True, description = "Show second derivative")
point_box = widgets.HBox([point_label, point_slider, show_deriv, show_second])

# Create derivative plot
deriv_fig = plt.figure(title="derivative", layout = fig_layout, fig_margin=fig_margin_dict)
yyy = df(xx)
deriv_curve = plt.plot(xx, yyy, fig = deriv_fig)
dylimits = plt.ylim(min(yyy), max(yyy))
dy_point = df(xpoint)
deriv_point = plt.plot([xpoint], [dy_point], colors = ['red'], marker = 'circle')
deriv_value = plt.plot(xx, [dy_point]*nPts, colors = ['green'])

# add derivative line to derivative figure:
m2 = d2f(xpoint)
b2 = dy_point - xpoint * m2
deriv2_line = lambda x: m2*x + b2
df2_line = plt.plot(xx, deriv2_line(xx), colors=['magenta'])

# Create second derivative plot
deriv2_fig = plt.figure(title="second derivative", layout = toggle_layout, fig_margin=fig_margin_dict, 
                       visible=False)

second_curve = plt.plot(xx, d2f(xx), fig = deriv2_fig)
d2y_point = d2f(xpoint)
second_point = plt.plot([xpoint], [d2y_point], colors = ['red'], marker = 'circle')
deriv2_value = plt.plot(xx, [d2y_point]*nPts, colors = ['magenta'])

# plot box
plot_box = widgets.HBox([main_fig, deriv_fig, deriv2_fig])

# function entry
func_label = widgets.Label("Enter function: ")
func_entry = widgets.Text(strFunc, continuous_update = False)
func_box = widgets.HBox([func_label, func_entry])

# domain entry
dom_layout = widgets.Layout(width = '75px')
dom_label = widgets.Label("Enter domain: ")
dom_low = widgets.FloatText(value = -3.0, layout = dom_layout)
dom_high = widgets.FloatText(value = 3.0, layout = dom_layout)
out_layout = widgets.Layout(border="blue solid 2px")
out_str = "The derivative at {:.4f} is {:.4f}"
output = widgets.Text(value = out_str.format(xpoint, dy_point), layout = out_layout)
dom_box = widgets.HBox([dom_label, dom_low, dom_high, output])

# callback functions
def update_plot(change):
    #if change['owner'] == func_entry:
    strFunc = func_entry.value
    f, df, d2f, fstr = set_function(strFunc)
    title_label.value = title_format.format(sym.latex(fstr))
    low = dom_low.value
    high = dom_high.value
    point_slider.min = low
    point_slider.max = high
    x = linspace(dom_low.value, dom_high.value, nPts)
    y = f(x)
    # fix y-axis limits:
    ylimits.min = min(y)
    ylimits.max = max(y)
    
    # update main plot
    function_curve.x = x
    function_curve.y = y
    df_line.x = x
    osc_curve.x = x
    
    xpoint = point_slider.value
    ypoint = f(xpoint)
    m = df(xpoint)
    b = ypoint - xpoint * m
    df_line.y = m*x + b
    point.x = [xpoint]
    point.y = [f(xpoint)]
    
    # update derivative plot
    deriv_curve.x = x
    deriv_curve.y = df(x)
    dylimits.min = min(deriv_curve.y)
    dylimits.max = max(deriv_curve.y)
    dypoint = m
    m2 = d2f(xpoint)
    b2 = m - xpoint * m2
    c = ypoint - xpoint * m + 0.5*xpoint**2*m2
    osc_curve.y = 0.5*m2*x**2 + b2*x + c
    df2_line.x = x
    df2_line.y = m2*x + b2
    deriv_point.x = [xpoint]
    deriv_point.y = [dypoint]
    deriv_value.x = x
    deriv_value.y = [df(xpoint)]*nPts
    
    # update second derivative plot
    second_curve.x = x
    second_curve.y = d2f(x)
    second_point.x = [xpoint]
    second_point.y = [d2f(xpoint)]
    deriv2_value.x = x
    deriv2_value.y = [d2f(xpoint)]*nPts
    
    # update output
    output.value = out_str.format(xpoint, df(xpoint))
    
def update_function(change):
    global f, df
    strFunc = func_entry.value
    f, df, fstr = set_function(strFunc)
    title_label.value = title_format.format(sym.latex(fstr))
    update_plot("called from update_function")
    
def toggle_vis(change):
    bShowFirst = show_deriv.value
    bShowSecond = show_second.value
    df_line.visible = bShowFirst
    deriv_value.visible = bShowFirst
    df2_line.visible = bShowSecond
    deriv2_value.visible = bShowSecond
    osc_curve.visible = bShowSecond
    
point_slider.observe(update_plot, names = 'value')
func_entry.observe(update_plot, names = 'value')
dom_low.observe(update_plot, names = 'value')
dom_high.observe(update_plot, names = 'value')
show_deriv.observe(toggle_vis, names = 'value')
show_second.observe(toggle_vis, names = 'value')

outbox = widgets.VBox([title_label, plot_box, point_box, func_box, dom_box])
outbox

In [ ]:
%%html
<style>
div.input{
    display:none;
}
</style>